# Создание формата .yaml и архитектуры данных для YOLO

In [8]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

csv_file_path = '/content/drive/MyDrive/data (1).csv'
data = pd.read_csv(csv_file_path)

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

In [10]:
import os
import shutil

base_dir = 'data'
os.makedirs(os.path.join(base_dir, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'images', 'val'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'labels', 'val'), exist_ok=True)

# Функция для преобразования данных в формат YOLO
def convert_to_yolo_format(row):
    img_width, img_height = 1920, 1080
    x_center = (row['x_left_bottom'] + row['length'] / 2) / img_width
    y_center = (row['y_left_bottom'] + row['width'] / 2) / img_height
    width = row['length'] / img_width
    height = row['width'] / img_height
    return f"{row['main_class']} {x_center} {y_center} {width} {height}\n"

for index, row in train_data.iterrows():
    shutil.copy(row['path'], os.path.join(base_dir, 'images', 'train', os.path.basename(row['path'])))

    label = convert_to_yolo_format(row)
    with open(os.path.join(base_dir, 'labels', 'train', os.path.basename(row['filename']).replace('.jpg', '.txt')), 'w') as f:
        f.write(label)

for index, row in val_data.iterrows():
    shutil.copy(row['path'], os.path.join(base_dir, 'images', 'val', os.path.basename(row['path'])))

    label = convert_to_yolo_format(row)
    with open(os.path.join(base_dir, 'labels', 'val', os.path.basename(row['path']).replace('.jpg', '.txt')), 'w') as f:
        f.write(label)

print("Структура папок и файлы меток успешно созданы!")

Структура папок и файлы меток успешно созданы!


In [11]:
import yaml

# Создание конфигурационного файла YAML
config = {
    'train': os.path.join(base_dir, 'images', 'train'),
    'val': os.path.join(base_dir, 'images', 'val'),
    'nc': 6,
    'names': ['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5']
}

with open('data.yaml', 'w') as yaml_file:
    yaml.dump(config, yaml_file)

print("Файл data.yaml успешно создан!")
# ПОСЛЕ СОЗДАНИЯ КОРРЕКТИРУЕТСЯ ПУТЬ (PATH)


Файл data.yaml успешно создан!


# Обновление базы данных

In [13]:
import os
import shutil
# Убираем ненужный класс из другой базы данных
def remove_depression_labels(images_dir, labels_dir):
    for label_file in os.listdir(labels_dir):
        label_path = os.path.join(labels_dir, label_file)
        with open(label_path, 'r') as file:
            lines = file.readlines()

        if any('2' in line for line in lines):
            os.remove(label_path)
            img_file = label_file.replace('.txt', '.jpg')
            img_path = os.path.join(images_dir, img_file)
            if os.path.exists(img_path):
                os.remove(img_path)

remove_depression_labels('/content/drive/MyDrive/2-defect-detection.v3i.yolov11/train/images', '/content/drive/MyDrive/2-defect-detection.v3i.yolov11/train/labels')
remove_depression_labels('/content/drive/MyDrive/2-defect-detection.v3i.yolov11/val/images', '/content/drive/MyDrive/2-defect-detection.v3i.yolov11/val/labels')


In [14]:
import os
import shutil
# Сливаем данные в одну базу
def transfer_data(src_images_dir, src_labels_dir, dest_images_dir, dest_labels_dir):
    os.makedirs(dest_images_dir, exist_ok=True)
    os.makedirs(dest_labels_dir, exist_ok=True)

    for image_file in os.listdir(src_images_dir):
        src_image_path = os.path.join(src_images_dir, image_file)
        dest_image_path = os.path.join(dest_images_dir, image_file)
        shutil.copy(src_image_path, dest_image_path)

    for label_file in os.listdir(src_labels_dir):
        src_label_path = os.path.join(src_labels_dir, label_file)
        dest_label_path = os.path.join(dest_labels_dir, label_file)
        shutil.copy(src_label_path, dest_label_path)

src_train_images = '/content/drive/MyDrive/2-defect-detection.v3i.yolov11/train/images'
src_train_labels = '/content/drive/MyDrive/2-defect-detection.v3i.yolov11/train/labels'
dest_train_images = '/content/data/images/train'
dest_train_labels = '/content/data/images/val'

src_val_images = '/content/drive/MyDrive/2-defect-detection.v3i.yolov11/val/images'
src_val_labels = '/content/drive/MyDrive/2-defect-detection.v3i.yolov11/val/labels'
dest_val_images = '/content/data/images/val'
dest_val_labels = '/content/data/labels/val'

transfer_data(src_train_images, src_train_labels, dest_train_images, dest_train_labels)

transfer_data(src_val_images, src_val_labels, dest_val_images, dest_val_labels)


# Обучение модели YOLO

In [1]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.7/882.7 kB 4.9 MB/s eta 0:00:00


In [ ]:
import os
from ultralytics import YOLO

data_yaml_path = '/content/drive/MyDrive/data/data (1).yaml'
epochs = 50

# Загрузка предобученной модели YOLOv11
model = YOLO("yolo11m.pt")

results = model.train(data=data_yaml_path, epochs=epochs, batch=-1)

# Сохранение модели после обучения
model.save('yolov11m_model_50.pt')

print("Обучение завершено! Модель сохранена как yolov11m_model_50.pt")

# Метрика mAP и получение предсказаний

In [24]:
import cv2
import numpy as np
from ultralytics import YOLO

# Функция для получения предсказаний из модели YOLOv11x
model = YOLO('/PATH/TO/yolov11x_custom_model_200.pt')

def get_yolo_predictions(image_path):
    image = cv2.imread(image_path)
    results = model.predict(source=image, conf=0.01)
    predictions = []

    for result in results:
        for box in result.boxes:
            x_min, y_min, x_max, y_max = box.xyxy[0].tolist()
            class_id = int(box.cls[0])
            confidence = box.conf[0].item()

            predictions.append([x_min, y_min, x_max, y_max, class_id, confidence])

    return predictions

imgs = ['PATH/TO/IMAGE1', 'PATH/TO/IMAGE2']
for img in imgs:
  pred = get_yolo_predictions(img)
  if len(pred) != 0:
      print(f'Координаты рамки:', {pred[0], pred[1], pred[2], pred[3]})
      print(f'ID класса:', {pred[4]})
      print(f'Уверенность модели:', {pred[5]})
  else:
      print('Проблем не обнаружено')


0: 640x480 (no detections), 2667.1ms
Speed: 4.5ms preprocess, 2667.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)
Проблем не обнаружено

0: 640x480 (no detections), 4556.1ms
Speed: 7.5ms preprocess, 4556.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)
Проблем не обнаружено


In [10]:
# Функция для загрузки истинных меток
def load_ground_truth(label_file):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    ground_truths = []
    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])

        # Преобразуем в форматы [x_min, y_min, x_max, y_max]
        x_min = x_center - width / 2
        y_min = y_center - height / 2
        x_max = x_center + width / 2
        y_max = y_center + height / 2

        ground_truths.append([x_min, y_min, x_max, y_max, class_id])
    return ground_truths

In [25]:
import os
import numpy as np
from sklearn.metrics import average_precision_score

def calculate_iou(box1, box2):
    """Вычисляет Intersection over Union (IoU) для двух прямоугольников."""
    x_min = max(box1[0], box2[0])
    y_min = max(box1[1], box2[1])
    x_max = min(box1[2], box2[2])
    y_max = min(box1[3], box2[3])

    intersection_area = max(0, x_max - x_min) * max(0, y_max - y_min)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    union_area = box1_area + box2_area - intersection_area
    return intersection_area / union_area if union_area > 0 else 0

def calculate_map(all_predictions, all_ground_truths, iou_threshold=0.5):
    """Вычисляет mAP для списка предсказаний и истинных меток."""
    # Сортируем предсказания по уверенности (предполагается, что предсказания содержат уверенность)
    all_predictions.sort(key=lambda x: x[4], reverse=True)  # Предполагается, что уверенность находится в 5-м элементе

    true_positives = np.zeros(len(all_predictions))
    false_positives = np.zeros(len(all_predictions))
    num_ground_truths = len(all_ground_truths)

    detected = set()  # Для отслеживания обнаруженных истинных меток

    for i, pred in enumerate(all_predictions):
        pred_box = pred[:4]  # [x_min, y_min, x_max, y_max]
        pred_class = pred[5]  # Класс предсказания

        # Проверяем, есть ли совпадение с истинными метками
        best_iou = 0
        best_gt_index = -1

        for j, gt in enumerate(all_ground_truths):
            if gt[4] == pred_class and j not in detected:  # Проверяем класс и не был ли обнаружен
                iou = calculate_iou(pred_box, gt[:4])
                if iou > best_iou:
                    best_iou = iou
                    best_gt_index = j

        if best_iou >= iou_threshold:
            true_positives[i] = 1
            detected.add(best_gt_index)
        else:
            false_positives[i] = 1

    # Вычисляем precision и recall
    cumulative_tp = np.cumsum(true_positives)
    cumulative_fp = np.cumsum(false_positives)

    precision = cumulative_tp / (cumulative_tp + cumulative_fp + 1e-6)  # Добавляем малое значение для избежания деления на ноль
    recall = cumulative_tp / num_ground_truths if num_ground_truths > 0 else 0

    # Вычисляем mAP
    ap = average_precision_score(y_true=[1] * num_ground_truths + [0] * len(all_predictions),
                                 y_score=list(precision) + list(precision))
    return ap


# Пример использования функции
test_images_dir = '/content/data/images/val'
test_labels_dir = '/content/data/labels/val'

all_predictions = []
all_ground_truths = []

for filename in os.listdir(test_images_dir):
    if filename.endswith('.jpg'):
        image_path = os.path.join(test_images_dir, filename)
        label_file = os.path.join(test_labels_dir, filename.replace('.jpg', '.txt'))

        # Получаем предсказания модели
        predictions = get_yolo_predictions(image_path)  # Предполагается, что эта функция возвращает предсказания в формате [x_min, y_min, x_max, y_max, confidence, class_id]

        # Загружаем истинные метки
        ground_truths = load_ground_truth(label_file)

        all_predictions.extend(predictions)
        all_ground_truths.extend(ground_truths)

# Вычисляем mAP
map_score = calculate_map(all_predictions, all_ground_truths)
print(f"Mean Average Precision (mAP): {map_score}")

Mean Average Precision (mAP): 0.8628129629
